In [1]:
import os
import numpy as np
from onnxruntime import InferenceSession

import tkinter as tk
from tkinterdnd2 import DND_FILES, TkinterDnD
from PIL import Image, ImageTk


IMG_HEIGHT, IMG_WIDTH = 64, 64
ONNX_PATH = "./dog_owl.onnx"
CLASSES = ["犬", "フクロウ"]

class MyApp(TkinterDnD.Tk):
    def __init__(self):
        super().__init__()

        self.is_imshow = False
        self.img_path = None

        # ウィンドウ設定
        self.title("Dog or Owl")
        self.geometry('500x500')
   
        # 画像表示用フレーム
        self.img_frame = tk.LabelFrame(self, width=400, height=400, text="ここにドラッグ&ドロップしてください", labelanchor="n")
        self.img_frame.drop_target_register(DND_FILES)
        self.img_frame.dnd_bind('<<Drop>>', self.funcDragAndDrop)
        self.img_frame.pack()

        # 分類結果表示用テキスト
        self.text1 = tk.Label(self, text="ここに分類結果が表示されます")
        self.text1.pack()
        self.text2 = tk.Label(self, text="")
        self.text2.pack()

    # 画像の分類
    def classify(self):
        if self.is_imshow:
            self.clear_image()

        # 画像の読み込み
        img = Image.open(open(self.img_path, 'rb'))
        img = img.convert('RGB')

        # 画像の分類
        x = img.resize((IMG_WIDTH, IMG_HEIGHT))
        x = np.array(x, dtype=np.float32)
        x = x[None, ...]
        model = InferenceSession(ONNX_PATH)
        input_name = model.get_inputs()[0].name
        result = model.run(['output'], {input_name: x})[0][0]
        class_id = np.argmax(result)
        class_name = CLASSES[class_id]

        # 画像の表示
        img_photo = ImageTk.PhotoImage(img.resize((400, 400)))
        self.img_view = tk.Label(self.img_frame, image=img_photo)
        self.img_view.image = img_photo
        self.img_view.pack()

        # テキストの更新
        self.text1['text'] = f"{os.path.basename(self.img_path)} は {class_name} の画像です"
        self.text2['text'] = f"犬である確率は {result[0]*100:.2f} %, フクロウである確率は {result[1]*100:.2f} %"

        self.is_imshow = True
        
    # 画像のクリア
    def clear_image(self):
        try:
            self.img_view.image = None
            self.img_view.destroy()
        except Exception as e:
            print(f"{e}")

        self.is_imshow = False

    # ドラッグ&ドロップによるファイルパスの取得
    def funcDragAndDrop(self, event):
        p = event.data
        p = p.strip('{').strip('}')
        if os.path.splitext(p)[1].lower() in ['.jpg', '.jpeg', '.png']:
            self.img_path = p
            self.classify()
        else:
            self.text1['text'] = f"{os.path.basename(p)} は画像ファイルではありません"
            self.text2['text'] = ""
            if self.is_imshow:
                self.clear_image()

if __name__ == "__main__":
    app = MyApp()
    app.mainloop()

[0.55824846 0.44175145]
[0.99058914 0.00941083]
[1.2080270e-05 9.9998796e-01]
[0.2720624  0.72793764]
[0.84490824 0.15509175]
[0.98904103 0.01095897]
[9.9944133e-01 5.5866496e-04]
[0.99212027 0.00787979]
[0.5986128  0.40138713]
[0.89177495 0.10822503]
[0.9761534  0.02384659]
[0.32556108 0.67443895]
[0.69623893 0.30376115]
[0.08884206 0.9111579 ]
[0.09708735 0.9029127 ]
[0.46238118 0.5376189 ]
[0.61798096 0.38201904]
[0.09732872 0.9026713 ]
[0.55589205 0.444108  ]
